<a href="https://colab.research.google.com/github/Nageswar-Sahoo/S4/blob/main/EVA_Session_4_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torchvision import datasets, transforms

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3) # i/p- 1*28*28  o/p- 32*26*26*
        self.batch1=nn.BatchNorm2d(32)   # i/p- 32*26*26  o/p- 32*26*26*
        self.pool1 = nn.MaxPool2d(2, 2)  # i/p- 32*26*26  o/p- 32*13*13*
        self.conv2 = nn.Conv2d(32, 32, 3)# i/p- 32*13*13  o/p- 32*11*11*
        self.batch2=nn.BatchNorm2d(32)   # i/p- 32*11*11  o/p- 32*11*11*
        self.pool2 = nn.MaxPool2d(2, 2)  # i/p- 32*11*11  o/p- 32*5*5*
        self.conv3 = nn.Conv2d(32, 16, 3)# i/p- 32*5*5  o/p- 32*3*3*
        self.batch3=nn.BatchNorm2d(16)   # i/p- 32*3*3  o/p- 32*3*3*
        self.droput1=nn.Dropout(0.25)    
        self.fc1 = nn.Linear(in_features=16 * 3 * 3, out_features=20)  # i/p- 16*3*3  o/p- 16*3*3*20
        self.batch4=nn.BatchNorm1d(20)  # i/p- 16*3*3  o/p- 16*3*3*

        self.droput2=nn.Dropout(0.10)    # i/p- 1*28*28  o/p- 32*26*26*
        self.fc2 = nn.Linear(in_features=20, out_features=10) # i/p- 20  o/p- 10

 

    def forward(self, x):
        x=self.conv1(x)
        x=F.relu(x)
        x=self.batch1(x)
        x=self.pool1(x)
        x=self.conv2(x)
        x=F.relu(x)
        x=self.batch2(x)
        x=self.pool2(x)     
        x=self.conv3(x)
        x=F.relu(x)
        x=self.batch3(x)   
        x=self.droput1(x)
        x = x.reshape(-1, 16 * 3 * 3)
        x = self.fc1(x) 
        x=F.relu(x)  
        x=self.batch4(x)   
        x=self.droput2(x)
        x = self.fc2(x)
        x=F.relu(x)
        return F.log_softmax(x)

In [10]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
       BatchNorm2d-2           [-1, 32, 26, 26]              64
         MaxPool2d-3           [-1, 32, 13, 13]               0
            Conv2d-4           [-1, 32, 11, 11]           9,248
       BatchNorm2d-5           [-1, 32, 11, 11]              64
         MaxPool2d-6             [-1, 32, 5, 5]               0
            Conv2d-7             [-1, 16, 3, 3]           4,624
       BatchNorm2d-8             [-1, 16, 3, 3]              32
           Dropout-9             [-1, 16, 3, 3]               0
           Linear-10                   [-1, 20]           2,900
      BatchNorm1d-11                   [-1, 20]              40
          Dropout-12                   [-1, 20]               0
           Linear-13                   [-1, 10]             210
Total params: 17,502
Trainable params: 

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [11]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [12]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.5f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [21]:

model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.3896883428096771 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.54it/s]



Test set: Average loss: 0.2988, Accuracy: 8840/10000 (88.40000%)



loss=0.3031056225299835 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 23.82it/s]



Test set: Average loss: 0.2825, Accuracy: 8859/10000 (88.59000%)



loss=0.11326312273740768 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 24.71it/s]


...


Test set: Average loss: 0.0290, Accuracy: 9919/10000 (99.19000%)



loss=0.054022740572690964 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 24.69it/s]



Test set: Average loss: 0.0307, Accuracy: 9912/10000 (99.12000%)



loss=0.04626404866576195 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 24.77it/s]



Test set: Average loss: 0.0287, Accuracy: 9924/10000 (99.24000%)



In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, 3) # i/p- 1*28*28  o/p- 32*26*26*
        self.batch1=nn.BatchNorm2d(32)   # i/p- 32*26*26  o/p- 32*26*26*
        self.pool1 = nn.MaxPool2d(2, 2)  # i/p- 32*26*26  o/p- 32*13*13*
        self.conv2 = nn.Conv2d(32, 32, 3)# i/p- 32*13*13  o/p- 32*11*11*
        self.batch2=nn.BatchNorm2d(32)   # i/p- 32*11*11  o/p- 32*11*11*
        self.pool2 = nn.MaxPool2d(2, 2)  # i/p- 32*11*11  o/p- 32*5*5*
        self.conv3 = nn.Conv2d(32, 16, 3)# i/p- 32*5*5  o/p- 32*3*3*
        self.batch3=nn.BatchNorm2d(16)   # i/p- 32*3*3  o/p- 32*3*3*
        self.droput1=nn.Dropout(0.25)    
        self.fc1 = nn.Linear(in_features=16 * 3 * 3, out_features=20)  # i/p- 16*3*3  o/p- 16*3*3*20
        self.batch4=nn.BatchNorm1d(20)  # i/p- 16*3*3  o/p- 16*3*3*
        self.droput2=nn.Dropout(0.10)    # i/p- 1*28*28  o/p- 32*26*26*
        self.fc2 = nn.Linear(in_features=20, out_features=10) # i/p- 20  o/p- 10



 

    def forward(self, x):
        x=self.conv1(x)
        x=F.relu(x)
        x=self.batch1(x)
        x=self.pool1(x)
        x=self.conv2(x)
        x=F.relu(x)
        x=self.batch2(x)
        x=self.pool2(x)     
        x=self.conv3(x)
        x=F.relu(x)
        x=self.batch3(x)   
        x=self.droput1(x)
        x = x.reshape(-1, 16 * 3 * 3)
        x = self.fc1(x) 
        x=F.relu(x)  
        x=self.batch4(x)   
        x=self.droput2(x)
        x = self.fc2(x)
        x=F.relu(x)
        return F.log_softmax(x)

In [25]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
       BatchNorm2d-2           [-1, 32, 26, 26]              64
         MaxPool2d-3           [-1, 32, 13, 13]               0
            Conv2d-4           [-1, 32, 11, 11]           9,248
       BatchNorm2d-5           [-1, 32, 11, 11]              64
         MaxPool2d-6             [-1, 32, 5, 5]               0
            Conv2d-7             [-1, 16, 3, 3]           4,624
       BatchNorm2d-8             [-1, 16, 3, 3]              32
           Dropout-9             [-1, 16, 3, 3]               0
           Linear-10                   [-1, 20]           2,900
      BatchNorm1d-11                   [-1, 20]              40
          Dropout-12                   [-1, 20]               0
           Linear-13                   [-1, 10]             210
Total params: 17,502
Trainable params: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [26]:
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.11460938304662704 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.27it/s]



Test set: Average loss: 0.0748, Accuracy: 9829/10000 (98.29000%)



loss=0.04542025923728943 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.46it/s]



Test set: Average loss: 0.0482, Accuracy: 9870/10000 (98.70000%)



loss=0.029887935146689415 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 24.80it/s]


...


Test set: Average loss: 0.0262, Accuracy: 9923/10000 (99.23000%)



loss=0.005192214157432318 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.34it/s]



Test set: Average loss: 0.0278, Accuracy: 9915/10000 (99.15000%)



loss=0.05722421780228615 batch_id=468: 100%|██████████| 469/469 [00:19<00:00, 24.31it/s]



Test set: Average loss: 0.0288, Accuracy: 9905/10000 (99.05000%)



In [ ]:
## With Global Average Pooling


In [20]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3) # i/p- 1*28*28  o/p- 32*26*26*
        self.batch1=nn.BatchNorm2d(32)   # i/p- 32*26*26  o/p- 32*26*26*
        self.pool1 = nn.MaxPool2d(2, 2)  # i/p- 32*26*26  o/p- 32*13*13*
        self.conv2 = nn.Conv2d(32, 32, 3)# i/p- 32*13*13  o/p- 32*11*11*
        self.batch2=nn.BatchNorm2d(32)   # i/p- 32*11*11  o/p- 32*11*11*
        self.pool2 = nn.MaxPool2d(2, 2)  # i/p- 32*11*11  o/p- 32*5*5*
        self.conv3 = nn.Conv2d(32, 16, 3)# i/p- 32*5*5  o/p- 32*3*3*
        self.batch3=nn.BatchNorm2d(16)   # i/p- 32*3*3*  o/p- 32*3*3*
        self.droput1=nn.Dropout(0.40)    # i/p- 32*3*3*  o/p- 32*3*3*
        self.fc1 = nn.Linear(in_features=16 * 1 * 1, out_features=128)  # i/p- 16*1*1  o/p- 128   With Global Average Pooling
        self.batch4=nn.BatchNorm1d(128)  # i/p- 128  o/p- 128
        self.droput2=nn.Dropout(0.20)    # i/p- 128  o/p- 128
        self.fc2 = nn.Linear(in_features=128, out_features=10) # i/p- 128  o/p- 10 

    def forward(self, x):
        x=self.conv1(x)
        x=F.relu(x)
        x=self.batch1(x)
        x=self.pool1(x)
        x=self.conv2(x)
        x=F.relu(x)
        x=self.batch2(x)
        x=self.pool2(x)     
        x=self.conv3(x)
        x=F.relu(x)
        x=self.batch3(x)   
        x=self.droput1(x)
        x = torch.mean(x.view(x.size(0), x.size(1), -1), dim=2)
        x = self.fc1(x) 
        x=F.relu(x)  
        x=self.batch4(x)   
        x=self.droput2(x)
        x = self.fc2(x)
        x=F.relu(x)
        return F.log_softmax(x)

In [21]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
       BatchNorm2d-2           [-1, 32, 26, 26]              64
         MaxPool2d-3           [-1, 32, 13, 13]               0
            Conv2d-4           [-1, 32, 11, 11]           9,248
       BatchNorm2d-5           [-1, 32, 11, 11]              64
         MaxPool2d-6             [-1, 32, 5, 5]               0
            Conv2d-7             [-1, 16, 3, 3]           4,624
       BatchNorm2d-8             [-1, 16, 3, 3]              32
           Dropout-9             [-1, 16, 3, 3]               0
           Linear-10                  [-1, 128]           2,176
      BatchNorm1d-11                  [-1, 128]             256
          Dropout-12                  [-1, 128]               0
           Linear-13                   [-1, 10]           1,290
Total params: 18,074
Trainable params: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [22]:
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.16647373139858246 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.99it/s]



Test set: Average loss: 0.0628, Accuracy: 9808/10000 (98.08000%)



loss=0.03898180276155472 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.84it/s]



Test set: Average loss: 0.0541, Accuracy: 9831/10000 (98.31000%)



loss=0.031198399141430855 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.91it/s]


...


Test set: Average loss: 0.0331, Accuracy: 9902/10000 (99.02000%)



loss=0.03489725664258003 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.60it/s]



Test set: Average loss: 0.0288, Accuracy: 9910/10000 (99.10000%)



loss=0.06774403899908066 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.58it/s]



Test set: Average loss: 0.0352, Accuracy: 9897/10000 (98.97000%)



In [40]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)  #I/P 1, 28, 28  op- 16, 26, 26]
        self.batch1=nn.BatchNorm2d(16) #16, 26, 26 op- 16, 26, 26
        self.conv2 = nn.Conv2d(16, 32, 3) # Ip- 16, 26, 26 op- 32, 24, 24
        self.batch2=nn.BatchNorm2d(32) # I/p 32, 24, 24 op- 32, 24, 24
        self.pool1 = nn.MaxPool2d(2, 2)  #Ip- 32, 24, 24 op- 32, 12, 12
        self.conv3 = nn.Conv2d(32, 10, 3)    # Ip- 32, 12, 12 O/p- 10, 10, 10     
        self.batch3=nn.BatchNorm2d(10) #I/P 10, 10, 10 op- 10, 10, 10
        self.pool2 = nn.MaxPool2d(2, 2) #  10, 10, 10 op-  10, 5, 5
        self.conv4 = nn.Conv2d(10, 10, 3) #10, 5, 5 op- 10, 3, 3
        self.droput1=nn.Dropout(0.20) #i/p  10, 3, 3 op -  10, 3, 3
        self.conv5 = nn.Conv2d(10, 10, 3) #ip  10, 3, 3 0p  10, 1, 1
 

    def forward(self, x):
        x=self.conv1(x)
        x=F.relu(x)
        x=self.batch1(x)
        x=self.conv2(x)
        x=F.relu(x)
        x=self.batch2(x)
        x=self.pool1(x)
        x=self.conv3(x)
        x=F.relu(x)
        x=self.batch3(x)
        x=self.droput1(x)       
        x=self.pool2(x)
        x=self.conv4(x)
        x=F.relu(x)
        x=self.droput1(x)
        x=self.conv5(x)
        x=F.relu(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [41]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
            Conv2d-3           [-1, 32, 24, 24]           4,640
       BatchNorm2d-4           [-1, 32, 24, 24]              64
         MaxPool2d-5           [-1, 32, 12, 12]               0
            Conv2d-6           [-1, 10, 10, 10]           2,890
       BatchNorm2d-7           [-1, 10, 10, 10]              20
           Dropout-8           [-1, 10, 10, 10]               0
         MaxPool2d-9             [-1, 10, 5, 5]               0
           Conv2d-10             [-1, 10, 3, 3]             910
          Dropout-11             [-1, 10, 3, 3]               0
           Conv2d-12             [-1, 10, 1, 1]             910
Total params: 9,626
Trainable params: 9,626
Non-trainable params: 0
-----------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [36]:


torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [37]:

model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.05918436497449875 batch_id=937: 100%|██████████| 938/938 [00:29<00:00, 31.40it/s]



Test set: Average loss: 0.0621, Accuracy: 9829/10000 (98.29000%)



loss=0.014312433078885078 batch_id=937: 100%|██████████| 938/938 [00:29<00:00, 31.80it/s]



Test set: Average loss: 0.0501, Accuracy: 9861/10000 (98.61000%)



loss=0.01721523143351078 batch_id=937: 100%|██████████| 938/938 [00:29<00:00, 31.94it/s]


...


Test set: Average loss: 0.0303, Accuracy: 9907/10000 (99.07000%)



loss=0.07346359640359879 batch_id=937: 100%|██████████| 938/938 [00:30<00:00, 31.19it/s]



Test set: Average loss: 0.0315, Accuracy: 9904/10000 (99.04000%)



loss=0.0006614286685362458 batch_id=937: 100%|██████████| 938/938 [00:29<00:00, 31.33it/s]



Test set: Average loss: 0.0320, Accuracy: 9898/10000 (98.98000%)



In [38]:

#With batch size 32 
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [39]:

model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.11398135125637054 batch_id=1874: 100%|██████████| 1875/1875 [00:42<00:00, 43.65it/s]



Test set: Average loss: 0.0613, Accuracy: 9812/10000 (98.12000%)



loss=0.031912583857774734 batch_id=1874: 100%|██████████| 1875/1875 [00:43<00:00, 43.42it/s]



Test set: Average loss: 0.0470, Accuracy: 9848/10000 (98.48000%)



loss=0.1543605476617813 batch_id=1874: 100%|██████████| 1875/1875 [00:43<00:00, 43.24it/s]


...


Test set: Average loss: 0.0315, Accuracy: 9906/10000 (99.06000%)



loss=0.0320684053003788 batch_id=1874: 100%|██████████| 1875/1875 [00:43<00:00, 42.63it/s]



Test set: Average loss: 0.0301, Accuracy: 9899/10000 (98.99000%)



loss=0.010425691492855549 batch_id=1874: 100%|██████████| 1875/1875 [00:43<00:00, 43.32it/s]



Test set: Average loss: 0.0306, Accuracy: 9901/10000 (99.01000%)

